# Imports

In [1]:
import numpy as np
from tensorflow.keras.models import load_model

import tkinter as tk
from tkinter import *

import win32gui

from PIL import ImageGrab, Image, ImageOps

In [2]:
#Let's load the model
model = load_model('mnist_model.h5')

# Preparation

Before building the GUI we should create functions to preprocess and predict our inputs.

Same Data preprocessing steps in "HWDrecognizer_model.ipynb"

* resize image to 28x28 px
* convert rgb to grayscale
* reshaping & normalizing to support our model input 

In [3]:
def to_digit(image):

    image = image.resize((28,28))
    image = image.convert('L')
    #Training data has a 'bg' of black and drawing of white
    #Our handwritten digit has a 'bg' of white and drawing of black
    #We should use ImageOps to invert pixel values
    image = ImageOps.invert(image)
    image = np.array(image)
    image = image.reshape(1,28,28,1)
    image = image/255.0
    
    digit = model.predict([image])[0]
    return np.argmax(digit), max(digit)

# Building The GUI

In [4]:
class Digitrecognizer(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        self.title("HWDrecognizer")
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="arrow")
        self.label = tk.Label(self, text="Write", font=("Helvetica", 36))
        self.classify_btn = tk.Button(self, text = "Check", command = self.recognizer)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.write)

    def clear_all(self):
        self.canvas.delete("all")
        
    def recognizer(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        a,b,c,d = rect
        rect=(a+4,b+4,c-4,d-4)
        frame = ImageGrab.grab(rect)

        digit, accuracy = to_digit(frame)
        self.label.configure(text='Your digit is: '+str(digit)+'\n\n'+'Accuracy: '+str(int(accuracy*100))+'%')

    def write(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

# Running our Application

In [5]:
run = Digitrecognizer()
mainloop()